# 蛋白质折叠

## 目标和前提条件

通过这个具有挑战性的蛋白质折叠问题来提升您的建模技能。我们将向您展示如何使用Gurobi Python API创建该问题的二元优化模型，并使用Gurobi优化器求解。

这个模型来自H. Paul Williams所著《数学规划中的模型构建》第五版第289-290页和344-345页的示例28。

这是一个高级建模示例，我们假设您已经掌握Python和Gurobi Python API，并且具有构建数学优化模型的进阶知识。通常，这些示例的目标函数和/或约束条件比较复杂，或需要使用Gurobi Python API的高级功能。

**下载代码库** <br /> 
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

**Gurobi许可证** <br /> 
为了正确运行此Jupyter笔记本，您必须拥有Gurobi许可证。如果您没有许可证，可以作为*商业用户*申请[评估许可证](https://www.gurobi.com/downloads/request-an-evaluation-license/)，或作为*学术用户*下载[免费许可证](https://www.gurobi.com/academia/academic-program-and-licenses/)。

---
## 问题描述

本Jupyter笔记本中描述的问题基于Forrester和Greenberg(2008)发表的一篇题为《计算生物学中的二次二元规划模型》的论文中讨论的分子生物学问题。该问题涉及蛋白质，它由氨基酸链组成。在这个问题中，氨基酸有两种形式：亲水性（喜水）和疏水性（厌水）。下图给出了这样一个链的示例，其中疏水性氨基酸用粗体标记。

![chain](chain.PNG)

这样的链会自然折叠，使尽可能多的疏水性氨基酸靠近在一起。下图给出了该链在二维平面上的一种折叠方式，新的匹配用虚线标记。我们的目标是预测最优折叠方式。

![folding](folding.PNG)

要解决这里提出的问题，我们必须为一个包含50个氨基酸的链找到最优折叠方式，其中在位置2、4、5、6、11、12、17、20、21、25、27、28、30、31、33、37、44和46处的氨基酸是疏水性的。

---
## 模型构建

### 集合和索引

$k \in A =\{1,2,...,50\}$：氨基酸链。

$i,j \in H =\{2,4,5,6,11,12,17,20,21,25,27,28,30,31,33,37,44,46\} \subseteq Aminoacids$：疏水性氨基酸的子集。

### 决策变量

$\text{match}_{i,j} \equiv x_{i,j} = 1$，当且仅当疏水性氨基酸$i$与氨基酸$j$匹配，对于所有疏水性氨基酸$i < j \in H$。这种匹配不包括因在链中相邻而预定义的匹配，即$j > i+1$。

$\text{fold}_{k} \equiv y_{k} = 1$，当且仅当在链中第$i$个和第$(i+1)$个氨基酸之间发生折叠。

### 约束条件

对于每对疏水性氨基酸$i$和$j$，我们可以将它们匹配，如果：
* 它们不相邻，即尚未匹配，
* 它们在链中之间有偶数个氨基酸，
* 在$i$和$j$之间恰好有一个折叠。

这导致以下约束条件：

1. $y_{k} + x_{i,j} \leq 1, \; \forall k \in A, (i,j) \in H, \; \text{such that} \; i \leq k < j, \; \text{and} \; 
k \neq (i+j-1)/2$
2. $x_{i,j} \leq y_{k}, \; \text{where} \; k = (i+j-1)/2$

令$\text{H_fold} = \{(i,j) \in H: x_{i,j} \leq y_{k}, \; k = (i+j-1)/2  \}$为能够通过折叠实现匹配的疏水性氨基酸集合。

### 目标函数
目标是最大化疏水性氨基酸的匹配数量。

$$
\sum_{i,j \in \text{H_fold}} x_{i,j}
$$

---
## Python实现

我们导入Gurobi Python模块。

In [ ]:
%pip install gurobipy 
# 注意：受限许可证不足以运行此笔记本，需要完整许可证

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# 使用 Python 3.11 和 Gurobi 11.0 测试通过

## 输入数据

In [ ]:
# 氨基酸列表和疏水性氨基酸列表

acids = [*range(1,51)]

h_phobic = [2,4,5,6,11,12,17,20,21,25,27,28,30,31,33,37,44,46]

## 预处理

In [ ]:
# 创建用于生成模型的数据结构
list_ij = []

# 可以匹配的疏水性氨基酸的索引
for i in h_phobic:
    for j in h_phobic:
        if j > i + 1:
            tp = i,j
            list_ij.append(tp)
            
ij = gp.tuplelist(list_ij)

###
list_ik1j = []

list_ik2j = []

for i,j in ij:
    for k in range(i,j):
        if (k == (i+j-1)/2  ):
            tp = i,j,k
            list_ik2j.append(tp)
        else:
            tp = i,j,k
            list_ik1j.append(tp)

# 类型2约束的索引
ik2j = gp.tuplelist(list_ik2j)

# 类型1约束的索引
ik1j = gp.tuplelist(list_ik1j)

# 通过折叠实现的匹配
list_ijfold = []

for i,j,k in ik2j:
    tp = i,j
    list_ijfold.append(tp)

ijfold = gp.tuplelist(list_ijfold)

## 模型部署
我们创建一个模型和决策变量。有两种类型的决策变量：确定要匹配哪些疏水性氨基酸的变量，以及确定蛋白质在哪个氨基酸处发生折叠的变量。

In [ ]:
model = gp.Model('ProteinFolding')

# 匹配变量
match = model.addVars(ij, vtype=GRB.BINARY, name="match")

# 折叠变量
fold = model.addVars(acids, vtype=GRB.BINARY, name="fold")

Using license file c:\gurobi\gurobi.lic


### 折叠和匹配约束

1. $y_{k} + x_{i,j} \leq 1, \; \forall k \in A, (i,j) \in H, \; \text{such that} \; i \leq k < j, \; \text{and} \; 
k \neq (i+j-1)/2$
2. $x_{i,j} \leq y_{k}, \; \text{where} \; k = (i+j-1)/2$

In [ ]:
# 约束条件1：

C1 = model.addConstrs( (fold[k] + match[i,j] <= 1 for i,j,k in ik1j ) , name='C1')

# 约束条件2：

C2 = model.addConstrs( ( match[i,j] <= fold[k]  for i,j,k in ik2j ) , name='C2')

### 目标函数

最大化疏水性氨基酸的匹配数。

In [ ]:
# 目标函数

model.setObjective(gp.quicksum(match[i,j] for i,j in ijfold) , GRB.MAXIMIZE )


In [ ]:
# 验证模型公式

model.write('ProteinFolding.lp')

# 运行优化引擎

model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2441 rows, 197 columns and 4882 nonzeros
Model fingerprint: 0x7a3a8e69
Variable types: 0 continuous, 197 integer (197 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 1596 rows and 105 columns
Presolve time: 0.02s
Presolved: 845 rows, 92 columns, 1779 nonzeros
Variable types: 0 continuous, 92 integer (92 binary)

Root relaxation: objective 3.200000e+01, 90 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   32.00000    0   89   -0.00000   32.00000      -     -    0s
H    0     0                       8.0000000   32.000

In [ ]:
# 输出报告

print(f"疏水性氨基酸匹配的最优数量: {model.objVal}")

print("_______________________________________")
print(f"疏水性氨基酸的最优匹配方案")
print("_______________________________________")

for i,j,k in ik2j:
    if (match[i,j].x > 0.5): 
        print(f"疏水性氨基酸 {i} 和 {j} 匹配，在氨基酸 {k} 处发生折叠。")

Optimal number of hydrophobic acids matchings: 10.0
_______________________________________
Optimal matching of hydrophobic acids.
_______________________________________
Hydrophobic acid matching (2, 5) with folding at amonacid 3.
Hydrophobic acid matching (5, 12) with folding at amonacid 8.
Hydrophobic acid matching (6, 11) with folding at amonacid 8.
Hydrophobic acid matching (12, 17) with folding at amonacid 14.
Hydrophobic acid matching (17, 20) with folding at amonacid 18.
Hydrophobic acid matching (20, 25) with folding at amonacid 22.
Hydrophobic acid matching (25, 28) with folding at amonacid 26.
Hydrophobic acid matching (28, 31) with folding at amonacid 29.
Hydrophobic acid matching (31, 46) with folding at amonacid 38.
Hydrophobic acid matching (33, 44) with folding at amonacid 38.


---
## 参考文献

H. Paul Williams，《数学规划中的模型构建》，第五版。

Forrester, R.J. 和 Greenberg, H.J. (2008) 计算生物学中的二次二元规划模型。算法运筹学，3，110–129。

版权所有 © 2020 Gurobi Optimization, LLC